In [1]:
## Import and data sources
import numpy as np                              
import pandas as pd                             
import urllib.request as req                    
import ast
import matplotlib.pyplot as plt            
import seaborn as sns                           
import sys
import os
import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)

# To supress warnings from shap
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
N_THREADS = 20  ## NOTE: The number of threads for parallel runs. Modify as needed.
os.environ['OMP_NUM_THREADS'] = str(N_THREADS)
    
# sns.set(font_scale=0.9, style='white')

census_key =  ## The key from https://www.census.gov/data/developers.html "Request a key".

# APIs from the US Census Bureau
population_density_url = 'https://api.census.gov/data/2019/pep/population?get=DENSITY&in=state:*&for=county:*&key={}'.format(census_key)
above_65_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S0101_C02_030E&in=state:*&for=county:*&key={}'.format(census_key)
transport_url = 'https://api.census.gov/data/2019/acs/acs5?get=GEO_ID,NAME,B08006_001E,B08006_003E,B08006_017E&in=state:*&for=county:*&key={}'.format(census_key)
race_url = 'https://api.census.gov/data/2019/acs/acs5/profile?get=GEO_ID,NAME,DP05_0070E,DP05_0077E&in=state:*&for=county:*&key={}'.format(census_key)
poverty_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S1701_C03_001E&in=state:*&for=county:*&key={}'.format(census_key)
income_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S1902_C03_019E&in=state:*&for=county:*&key={}'.format(census_key)
employed_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S2301_C03_001E&in=state:*&for=county:*&key={}'.format(census_key)
unemployment_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S2301_C04_001E&in=state:*&for=county:*&key={}'.format(census_key)
mean_commute_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_046E&in=state:*&for=county:*&key={}'.format(census_key)
labour_url = 'https://api.census.gov/data/2019/acs/acs5/profile?get=GEO_ID,NAME,DP03_0028PE,DP03_0030PE,DP03_0031PE&in=state:*&for=county:*&key={}'.format(census_key)
insurance_url = 'https://api.census.gov/data/2019/acs/acs5/profile?get=GEO_ID,NAME,DP03_0099PE&in=state:*&for=county:*&key={}'.format(census_key)
gini_url = 'https://api.census.gov/data/2019/acs/acs5?get=GEO_ID,NAME,B19083_001E&in=state:*&for=county:*&key={}'.format(census_key)
education_url = 'https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S1501_C02_012E&in=state:*&for=county:*&key={}'.format(census_key)

# links to the JHU GitHub
jhu_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
jhu_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

# comorbidities data stored locally
ccc_data = '../data/CDC/CDC_PMID_32233970.xlsx'
any_data = '../data/CDC/CDC_MMWR_6929.xlsx'

nensemble = 250

In [2]:
def sectorAnalyze(df, df_c, useDensity=False, type=0, scale=False, N=100, run_BDT=False):
    ''' A function to prepare the data for the BDT analysis
        arguments:
            df: the dataframe for the socio-economic conditions
            df_c: the dataframe for covid-19 prevalence
            useDensity: boolean to determine whether density is used in the analysis
            type:
                0: death rate
                1: confirmed case rate
                2: fatality rate
            scale: boolean to toggle scaling of the features
            N: The number of BDTs in an ensemble
            run_BDT: boolean to determin whether to call run_BDT() after cleaning the data
        return:
            df: the combined dataframe
    '''
    df = pd.merge(df, df_c, on='FIPS')
    if type == 0: df['death rate'] = np.log10(df['deaths'].clip(lower=0) / df['Population'] * 100000. + 1.)
    if type == 1: df['confirmed rate'] = np.log10(df['confirmed'].clip(lower=0) / df['Population'] * 100000. + 1.)
    if type == 2: df['fatality rate'] = df['deaths'].clip(lower=0) / df['confirmed_base'] * 100
    if useDensity: df.drop(columns=['Population', 'deaths', 'confirmed', 'confirmed_base'], inplace=True)
    else: df.drop(columns=['Population', 'deaths', 'confirmed', 'confirmed_base', 'Density'], inplace=True)

    return df

def makeCorrelationPlot(df, df_c, filename='', cbar=False, only_corr=True):
    ''' A function to make the correlation plot using sns
        arguments:
            df: the dataframe to make the plot from
            cbar: boolean to decide whether to plot the colourbar
        return:
            fig: the figure
            ax: the axes
    '''
    df_plot = df.copy(deep=True)
    df_plot_0 = sectorAnalyze(df_plot, df_c, useDensity=True, type=0, run_BDT=False)
    df_plot_1 = sectorAnalyze(df_plot, df_c, useDensity=True, type=1, run_BDT=False)
    df_plot_2 = sectorAnalyze(df_plot, df_c, useDensity=True, type=2, run_BDT=False)
    df_plot_2.drop(df_plot_2[df_plot_2['fatality rate']==np.inf].index, inplace=True)
    df_plot = df_plot_1.copy(deep=True)
    df_plot['death rate'] = df_plot_0['death rate']
    df_plot['fatality rate'] = df_plot_2['fatality rate']
    df_plot.Density = 10**df_plot.Density
    df_plot['confirmed rate'] = 10**df_plot['confirmed rate'] - 1.
    df_plot['death rate'] = 10**df_plot['death rate'] - 1.
    df_plot['fatality rate'] = df_plot['fatality rate']
    
    if only_corr:
        return df_plot.corr(method='pearson')
    else:
        fig, ax = plt.subplots(figsize=(15.5, 13))
        mask = np.zeros((df_plot.shape[1],df_plot.shape[1]))
        mask[np.triu_indices_from(mask)] = True
        np.fill_diagonal(mask,0)
        cmap = sns.diverging_palette(220, 10, as_cmap=True)
        cbar_kws = {"aspect": 50, "ticks": [-1.0,-0.8,-0.6,-0.4,-0.2,0.0,0.2,0.4,0.6,0.8,1.0]}
        corr = df_plot.corr(method='pearson')
        sns.heatmap(round(corr, 2)+0., mask=mask, annot=True, cmap=cmap, vmin=-1.0, vmax=1.0, cbar_kws=cbar_kws, cbar=cbar, annot_kws={"fontsize":10})

        ax = plt.gca()
        ax.tick_params(labelsize=12)
        fig.set_size_inches(9.,9.)
        plt.tight_layout()
        if filename != '': plt.savefig(filename)
        return fig, ax, corr

def abs_shap(df_shap, df, lim=0.5):
    ''' A function to plot the bar plot for the mean abs SHAP values
        arguments:
            df_shap: the dataframe of the SHAP values
            df: the dataframe for the feature values for which the SHAP values have been determined
    '''
    # Make a copy of the input data
    shap_v = pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)

    # Determine the correlation in order to plot with different colors
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'#da3b46','#3f7f93')

    # Plot it
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(9,6),legend=False, zorder=2)
    ax.set_xlabel("Mean(|SHAP Value|) (Red = Positive Impact, Blue = Negative Impact)")
    ax.set_ylabel('')
    # Switch off ticks
    ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")
    plt.xlim(0, lim)
    # Draw vertical axis lines
    vals = ax.get_xticks()
    for tick in vals:
        ax.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#808080', zorder=1)
    
def get_census_data(url):
    ''' A function to get the data from USA Census Bureau
        arguments:
            url: the url for the API call
        return:
            df: the dataframe with the pulled data
    '''
    df = pd.DataFrame(ast.literal_eval(req.urlopen(req.Request(url)).read().decode('utf8').replace('null', '"1"')))
    df.columns = df.iloc[0]
    df = df[1:]
    df['FIPS'] = df.pop('GEO_ID').apply(lambda x: int(x[-5:]))
    return df

sns.set(font_scale=1.25, style='white')

In [3]:
## The density data from census.gov. We are removing the counties with density = 0 (Puerto Rico)
density = pd.DataFrame(ast.literal_eval(req.urlopen(req.Request(population_density_url)).read().decode('utf8').replace('null', '"0"')))
density.columns = density.iloc[0]
density = density[1:]
density.DENSITY = density.DENSITY.astype('float32')
density.state = density.state.astype('string')
density.county = density.county.astype('string')
density['FIPS'] = density['state'] + density['county']
density.FIPS = density.FIPS.astype('int32')
density = density.drop(labels=['state', 'county'], axis=1)
columns = density.columns.tolist()
columns = columns[::-1]
density = density[columns]
density.drop(density[density['DENSITY'] == 0].index , inplace=True)
density.DENSITY = np.log10(density.DENSITY)
density.rename(columns={'DENSITY': 'Density'}, inplace=True)

In [4]:
## The JHU data
def getJHUdata(date, basedate, cut=True):
    deaths = pd.read_csv(jhu_deaths)
    deaths.drop(deaths[(deaths['FIPS'] < 1000) | (deaths['FIPS'] > 57000)].index , inplace=True)
    deaths.dropna(inplace=True)

    confirmed = pd.read_csv(jhu_confirmed)
    confirmed.drop(confirmed[(confirmed['FIPS'] < 1000) | (confirmed['FIPS'] > 57000)].index , inplace=True)
    confirmed.dropna(inplace=True)

    deaths_df = deaths[['FIPS', 'Population', date[0], date[1]]]
    deaths_df[date[1]] = deaths_df[date[1]] - deaths_df[date[0]]
    deaths_df.drop(columns=[date[0]], inplace=True)
    
    confirmed_df = confirmed[['FIPS', basedate[0], basedate[1], date[0], date[1]]]
    confirmed_df[date[1]] = confirmed_df[date[1]] - confirmed_df[date[0]]
    confirmed_df[basedate[1]] = confirmed_df[basedate[1]] - confirmed_df[basedate[0]]
    confirmed_df.drop(columns=[date[0], basedate[0]], inplace=True)

    ## Death rate computation with the JHU data
    df_covid19 = deaths_df.copy()
    df_covid19.rename(columns={df_covid19.columns[-1]: 'deaths'}, inplace=True)
    df_covid19['confirmed'] = confirmed_df[date[1]]
    df_covid19['confirmed_base'] = confirmed_df[basedate[1]]
    df_covid19.dropna(inplace=True)
    df_covid19.reset_index(drop=True, inplace=True)

    ## Cleaning out low statistics data and merging JHU with density data
    if cut: 
        df_cut = df_covid19.drop(df_covid19[(df_covid19['confirmed'] < 1) & (df_covid19['deaths'] < 1)].index)
        df_cut.reset_index(drop=True, inplace=True)
        return df_cut
    else:
        return df_covid19

In [5]:
# The FIPS and state names
df_census = get_census_data(above_65_url)
df_census = df_census[['FIPS', 'NAME']]
df_census['State'] = df_census['NAME'].str.split(', ').str[1]
df_census['County'] = df_census.pop('NAME').str.split(', ').str[0]
df_census = df_census.sort_values(df_census.columns[0], ignore_index=True)
df_census = pd.merge(df_census, density, on='FIPS')

# 2019 5 year ACS data on Age
dft = get_census_data(above_65_url)
dft = dft[['FIPS', 'S0101_C02_030E']]
dft.rename(columns={'S0101_C02_030E': 'Senior Citizen'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Senior Citizen'] = df_census['Senior Citizen'].apply(lambda x: float(x))/100.

# 2019 5 year ACS data on transport used
dft = get_census_data(transport_url)
dft['Transit-1'] = 1. - (dft['B08006_003E'].astype('float')+dft['B08006_017E'].astype('float'))/dft['B08006_001E'].astype('float')
dft = dft[['FIPS', 'Transit-1']]
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Transit'] = df_census.pop('Transit-1')
df_census.at[df_census[df_census['FIPS']==35039].index, 'Transit'] = 0.159 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on non-white
dft = get_census_data(race_url)
# dft['Non-White'] = 1. - dft['B02001_002E'].astype('float')/dft['B02001_001E'].astype('float')
dft['Non-White'] = 1. - dft['DP05_0077E'].astype('float')/dft['DP05_0070E'].astype('float')
dft = dft[['FIPS', 'Non-White']]
df_census = pd.merge(df_census, dft, on='FIPS')

# 2019 5 year ACS data on Poverty
dft = get_census_data(poverty_url)
dft = dft[['FIPS', 'S1701_C03_001E']]
dft.rename(columns={'S1701_C03_001E': 'Poverty-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Poverty'] = df_census.pop('Poverty-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Poverty'] = 0.237 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on Income per capita
dft = get_census_data(income_url)
dft = dft[['FIPS', 'S1902_C03_019E']]
dft.rename(columns={'S1902_C03_019E': 'Income/Capita'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Income/Capita'] = df_census['Income/Capita'].apply(lambda x: float(x))
df_census.at[df_census[df_census['FIPS']==35039].index, 'Income/Capita'] = 19678 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on Employment
dft = get_census_data(employed_url)
dft = dft[['FIPS', 'S2301_C03_001E']]
dft.rename(columns={'S2301_C03_001E': 'Employed-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Employed'] = df_census.pop('Employed-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Employed'] = 0.382287 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on Unemployment
dft = get_census_data(unemployment_url)
dft = dft[['FIPS', 'S2301_C04_001E']]
dft.rename(columns={'S2301_C04_001E': 'Unemployment-1'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Unemployment'] = df_census.pop('Unemployment-1').apply(lambda x: float(x))/100.
df_census.at[df_census[df_census['FIPS']==35039].index, 'Unemployment'] = 0.114 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on Mean Commute in minutes
dft = get_census_data(mean_commute_url)
dft = dft[['FIPS', 'S0801_C01_046E']]
dft.rename(columns={'S0801_C01_046E': 'MeanCommute'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['MeanCommute'] = df_census['MeanCommute'].apply(lambda x: float(x))
df_census.at[df_census[df_census['FIPS']==35039].index, 'MeanCommute'] = 26.2 ## Imputed value from 2011 since values are null in 2019
df_census.at[df_census[df_census['FIPS']==48301].index, 'MeanCommute'] = 25.4 ## Imputed value from 2011 since values are null in 2019

# 2019 5 year ACS data on Labour
dft = get_census_data(labour_url)
dft['Labour'] = (dft['DP03_0028PE'].astype('float') + dft['DP03_0030PE'].astype('float') + dft['DP03_0031PE'].astype('float'))/100.
dft = dft[['FIPS', 'Labour']]
df_census = pd.merge(df_census, dft, on='FIPS')
df_census.at[df_census[df_census['FIPS']==35039].index, 'Labour'] = 0.271 + 0.108 + 0.07 ## Imputed value from 2011 since values are null in 2019

# 2018 5 year ACS data on Health Insurance
dft = get_census_data(insurance_url)
dft = dft[['FIPS', 'DP03_0099PE']]
dft.rename(columns={'DP03_0099PE': 'Uninsured'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Uninsured'] = df_census.pop('Uninsured').apply(lambda x: float(x))/100.

# 2019 5 year ACS data on Gini Index
dft = get_census_data(gini_url)
dft = dft[['FIPS', 'B19083_001E']]
dft.rename(columns={'B19083_001E': 'Gini'}, inplace=True)
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Gini'] = df_census['Gini'].astype('float')
df_census.at[df_census[df_census['FIPS']==35039].index, 'Gini'] = 0.4290 ## Imputed value from 2011 since values are null in 2019

# 2018 5 year ACS data on Health Insurance
dft = get_census_data(education_url)
dft = dft[['FIPS', 'S1501_C02_012E']] ##
dft.rename(columns={'S1501_C02_012E': 'Education'}, inplace=True) ##
df_census = pd.merge(df_census, dft, on='FIPS')
df_census['Education'] = df_census.pop('Education').apply(lambda x: float(x))/100.

In [6]:
# Comorbidities data CCC
df_ccc_data = pd.read_excel(ccc_data, engine='openpyxl')
df_ccc = df_ccc_data[['FIPS', 'ccc_measure', 'Beneficiaries', 'FFS_bene', 'cntypop', 'urban_code']]
df_ccc.drop(df_ccc_data[df_ccc_data['FIPS'] == 51515].index, inplace=True) # Bedford City has been merged with Bedford County
df_ccc.at[df_ccc[df_ccc['FIPS']==2270].index, 'FIPS'] = 2158 # Wade Hampton Census Area is now Kusilvak Census Area
df_ccc.at[df_ccc[df_ccc['FIPS']==46113].index, 'FIPS'] = 46102 # Shannon County is now Oglala Lakota County

# Comorbidities data any-condition
df_any_data = pd.read_excel(any_data, engine='openpyxl')
df_any = df_any_data[['FIPS', 'anycondition_prevalence']]

df_census = pd.merge(df_census, df_ccc, on='FIPS')
df_census = pd.merge(df_census, df_any, on='FIPS')

In [7]:
# Collecting all the necessary features into one dataframe
df_census_curated = pd.DataFrame()
df_census_curated['FIPS'] = df_census['FIPS']
df_census_curated['State'] = df_census['State']
df_census_curated['Density'] = df_census['Density']
df_census_curated['Non-White'] = df_census['Non-White']
df_census_curated['Poverty'] = df_census['Poverty']
df_census_curated['Income'] = df_census['Income/Capita']
df_census_curated['Unemployment'] = df_census['Unemployment']
df_census_curated['Uninsured'] = df_census['Uninsured']
df_census_curated['Employed'] = df_census['Employed']
df_census_curated['Labor'] = df_census['Labour']
df_census_curated['Transit'] = df_census['Transit']
df_census_curated['Mean Commute'] = df_census['MeanCommute']
df_census_curated['Senior Citizen'] = df_census['Senior Citizen']
df_census_curated['Gini'] = df_census['Gini']
# df_census_curated['Education'] = df_census['Education']
# df_census_curated['CCC'] = df_census['ccc_measure']
df_census_curated['Comorbidities'] = df_census['anycondition_prevalence'].apply(lambda x: float(x))/100.

_____________
## The East Coast

In [8]:
# Filtering out the states to be analysed.
states = ['District of Columbia', 'New Jersey', 'Rhode Island', 'Massachusetts', 'Connecticut', 'Maryland', 'Delaware', 'New York']

df_density_curated = pd.DataFrame()
for state in states:
    df_density_curated = pd.concat([df_density_curated, df_census_curated[df_census_curated['State'] == state]])
df_density_curated.drop(columns='State', inplace=True)


# Generating the plot for the correlation matrix for February to July
df_cut = getJHUdata(['2/15/20', '7/15/20'], ['2/5/20', '7/5/20'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/East_Coast_February_July.csv')

df_10 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_11 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_11['death rate'] = df_10['death rate']
df_11.to_csv('../data/East_Coast_February_July.csv')

# Generating the plot for the correlation matrix for July to January
df_cut = getJHUdata(['7/15/20', '1/15/21'], ['7/5/20', '1/5/21'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/East_Coast_July_January.csv')

df_20 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_21 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_21['death rate'] = df_20['death rate']
df_21.to_csv('../data/East_Coast_July_January.csv')

_______________
## The Southern States

In [9]:
# Filtering out the states to be analysed.
states = ['Alabama', 'Arkansas', 'Florida', 'Georgia', 'Kentucky', 'Louisiana',
          'Mississippi', 'North Carolina', 'Oklahoma', 'South Carolina', 'Tennessee', 'Texas', 'Virginia',
          'West Virginia']

df_density_curated = pd.DataFrame()
for state in states:
    df_density_curated = pd.concat([df_density_curated, df_census_curated[df_census_curated['State'] == state]])
df_density_curated.drop(columns='State', inplace=True)


# Generating the plot for the correlation matrix from February to July
df_cut = getJHUdata(['2/15/20', '7/15/20'], ['2/5/20', '7/5/20'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/Southern_States_February_July.csv')

df_10 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_11 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_11['death rate'] = df_10['death rate']
df_11.to_csv('../data/Southern_States_February_July.csv')

# Generating the plot for the correlation matrix for July to January
df_cut = getJHUdata(['7/15/20', '1/15/21'], ['7/5/20', '1/5/21'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/Southern_States_July_January.csv')

df_20 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_21 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_21['death rate'] = df_20['death rate']
df_21.to_csv('../data/Southern_States_July_January.csv')

_______________
## The West Coast

In [10]:
# Filtering out the states to be analysed.
states = ['California', 'Oregon', 'Washington']

df_density_curated = pd.DataFrame()
for state in states:
    df_density_curated = pd.concat([df_density_curated, df_census_curated[df_census_curated['State'] == state]])
df_density_curated.drop(columns='State', inplace=True)


# Generating the plot for the correlation matrix from February to July
df_cut = getJHUdata(['2/15/20', '7/15/20'], ['2/5/20', '7/5/20'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/West_Coast_February_July.csv')

df_10 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_11 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_11['death rate'] = df_10['death rate']
df_11.to_csv('../data/West_Coast_February_July.csv')

# Generating the plot for the correlation matrix for July to January
df_cut = getJHUdata(['7/15/20', '1/15/21'], ['7/5/20', '1/5/21'])
corr = makeCorrelationPlot(df_density_curated, df_cut)
corr.to_csv('../data/Correlations/West_Coast_July_January.csv')

df_20 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=0)
df_21 = sectorAnalyze(df_density_curated, df_cut, useDensity=True, type=1)
df_21['death rate'] = df_20['death rate']
df_21.to_csv('../data/West_Coast_July_January.csv')

__________________
## Panel Data (Weekly)

In [11]:
def getPanel(df_c, period, region, norm=True, weekly=False, datatype='confirmed_cases'):
    confirmed = pd.read_csv(jhu_confirmed).drop(columns=['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'])
    deaths = pd.read_csv(jhu_deaths).drop(columns=['UID', 'iso2', 'iso3', 'code3', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'])
    if weekly: 
        confirmed = pd.concat([confirmed['FIPS'], confirmed.drop(columns=['FIPS']).diff(axis=1).rolling(window=7, axis=1).sum().dropna(axis=1)], axis=1)
        deaths = pd.concat([deaths[['FIPS', 'Population']], deaths.drop(columns=['FIPS', 'Population']).diff(axis=1).rolling(window=7, axis=1).sum().dropna(axis=1)], axis=1)

    if datatype == 'confirmed_cases':
        df = pd.merge(confirmed, deaths[['FIPS', 'Population']], on='FIPS')
    elif datatype == 'death_rate':
        df = deaths
        
    df_1 = df.pop('FIPS')
    if norm: df = df.div(df.Population/100000, axis=0)
    df = pd.concat([df_1, df], axis=1).drop(columns=['Population'])
    df = df.dropna()[['FIPS']+period].set_index('FIPS').stack()
    
    
    panel = df_c.set_index('FIPS').join(pd.DataFrame(df, columns=[datatype]), how='inner').reset_index(level=1)
    panel['week'] = (pd.to_datetime(panel.level_1).dt.week - 7) % 53 + 7
    panel = panel.drop(columns='level_1')
    panel['region'] = region
    return panel

def mergePanels(df_c, period, norm=True, weekly=False, datatype='confirmed_cases'):
    # Filtering out the East Coast States.
    states_list = [['District of Columbia', 'New Jersey', 'Rhode Island', 'Massachusetts', 'Connecticut', 'Maryland', 'Delaware', 'New York'],
                   ['Alabama', 'Arkansas', 'Florida', 'Georgia', 'Kentucky', 'Louisiana',
                    'Mississippi', 'North Carolina', 'Oklahoma', 'South Carolina', 'Tennessee', 'Texas', 'Virginia',
                    'West Virginia'],
                   ['California', 'Oregon', 'Washington']]
    regions = ['east', 'south', 'west']
    
    panel_list = []
    for i in range(len(regions)):
        df_density_curated = pd.DataFrame()
        for state in states_list[i]:
            df_density_curated = pd.concat([df_density_curated, df_c[df_c['State'] == state]])
        df_density_curated.drop(columns='State', inplace=True)

        panel_list.append(getPanel(df_density_curated, period=period, region=regions[i], norm=norm, weekly=weekly, datatype=datatype))

    return pd.concat(panel_list)

### Normalized Weekly Cases (confirmed cases)

### Period I

In [12]:
# Set up the weeks
per_1 = []
for w in range(22):
    week = 6 + w
    year = 2020
    date = datetime.date(year, 1, 1) + relativedelta(weeks=+week) + relativedelta(days=+5)
    per_1.append(date.strftime("%-m/%-d/%y"))
    
panel_1 = mergePanels(df_census_curated, per_1, weekly=True)
panel_1.to_csv('../data/panel_merged_period_1_weekly_norm_diff_confirmed.csv')

### Period II

In [13]:
# Set up the weeks
per_2 = []
for w in range(22, 48):
    week = 6 + w
    year = 2020
    date = datetime.date(year, 1, 1) + relativedelta(weeks=+week) + relativedelta(days=+5)
    per_2.append(date.strftime("%-m/%-d/%y"))
    
panel_2 = mergePanels(df_census_curated, per_2, weekly=True)
panel_2.to_csv('../data/panel_merged_period_2_weekly_norm_diff_confirmed.csv')

### Normalized Weekly Cases (death rate)

### Period I

In [14]:
# Set up the weeks
per_1 = []
for w in range(22):
    week = 6 + w
    year = 2020
    date = datetime.date(year, 1, 1) + relativedelta(weeks=+week) + relativedelta(days=+5)
    per_1.append(date.strftime("%-m/%-d/%y"))
    
panel_1 = mergePanels(df_census_curated, per_1, weekly=True, datatype='death_rate')
panel_1.to_csv('../data/panel_merged_period_1_weekly_norm_diff_deaths.csv')

### Period II

In [15]:
# Set up the weeks
per_2 = []
for w in range(22, 48):
    week = 6 + w
    year = 2020
    date = datetime.date(year, 1, 1) + relativedelta(weeks=+week) + relativedelta(days=+5)
    per_2.append(date.strftime("%-m/%-d/%y"))
    
panel_2 = mergePanels(df_census_curated, per_2, weekly=True, datatype='death_rate')
panel_2.to_csv('../data/panel_merged_period_2_weekly_norm_diff_deaths.csv')